### Importing Libraries

In [1]:
import numpy as np
import pandas as pd
import re
from nltk.tokenize import word_tokenize
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

### Loading the Model

In [2]:
model = load_model('intent-classifier.h5')

In [3]:
def load_dataset(filename):
    # Reading the custom excel file designed for DURHAM COLLEGE COVID AWARENESS
    df = pd.read_excel(filename, names = ['Questions', 'Intents', 'Answer'])
    # Extract the Intents, Answers, and Questions
    intent = df['Intents']
    answer = df['Answer']
    # Extract unique Intents and Answers and zip them in Key-Value Pairs
    unique_intent = list(set(intent))
    unique_answers = list(set(answer))
    intent_answer_dict = dict(zip(unique_intent, unique_answers))
    questions = list(df['Questions'])

    return (intent, unique_intent, questions, intent_answer_dict)

In [4]:
intent, unique_intent, sentences, intent_answer_dict = load_dataset('covid-intents.xlsx')

### Data Preprocessing

In [5]:
def cleaning(sentences):
    words = []
    # Extract the sentences 
    for s in sentences:
        # Remove the punctuations
        clean = re.sub(r'[^ a-z A-Z 0-9]', " ", s)
        # Extract the words from the sentence
        w = word_tokenize(clean)
        # Converting the words to Lower case and converting them back to a list
        words.append([i.lower() for i in w])

    return words

In [6]:
cleaned_words = cleaning(sentences)

In [7]:
def create_tokenizer(words, filters = '!"#$%&()*+,-./:;<=>?@[\]^_`{|}~'):
    # Create a tokenizer with the filters defined above
    token = Tokenizer(filters = filters)
    # Fit the created tokenizer on the sentences
    token.fit_on_texts(words)
    return token

In [8]:
def max_length(words):
    # Get the Maximum length of all the sentences
    return(len(max(words, key = len)))

In [9]:
word_tokenizer = create_tokenizer(cleaned_words)
vocab_size = len(word_tokenizer.word_index) + 1
max_length = max_length(cleaned_words)

In [10]:
def encoding_doc(token, words):
    # Encoding the list of words to list of integers based on BoW
    return(token.texts_to_sequences(words))

In [11]:
# Encoding the sentences
encoded_doc = encoding_doc(word_tokenizer, cleaned_words)

In [12]:
def padding_doc(encoded_doc, max_length):
    # Padding the sequence of integers to max-len of sentences
    return(pad_sequences(encoded_doc, maxlen = max_length, padding = "post"))

In [13]:
# padding the Sequences
padded_doc = padding_doc(encoded_doc, max_length)

In [14]:
# Function to Preprocess the manual user inputs
def predictions(text):
    # Remove Punctuations
    clean = re.sub(r'[^ a-z A-Z 0-9]', " ", text)
    # Extract the words from the sentence
    test_word = word_tokenize(clean)
    # Converting the words to Lower case and converting them back to a list
    test_word = [w.lower() for w in test_word]
    # Encode sentences to sequences
    test_ls = word_tokenizer.texts_to_sequences(test_word)
    print(test_word)
    # Check for unknown words not in the actual BoW
    if [] in test_ls:
        test_ls = list(filter(None, test_ls))

    test_ls = np.array(test_ls).reshape(1, len(test_ls))
    # Pad the sequences to max length
    x = padding_doc(test_ls, max_length)
    # Predict the Intent
    pred = model.predict_proba(x)


    return pred

In [15]:
def get_final_output(pred, classes):
    # Get the Prediction scores
    predictions = pred[0]
    # Get all the unique Intents
    classes = np.array(classes)
    # Sort the prediction scores for all Intents
    ids = np.argsort(-predictions)
    # Sort all the Intents according to the scores using sorted indexes
    classes = classes[ids]
    predictions = -np.sort(-predictions)
    # Select the Top Intent
    predicted_intent = classes[0]
    # Select the Answer for the Intent selected (Key-value pair)
    return intent_answer_dict[predicted_intent]

In [16]:
text = "I would like to get some technical assistance."
pred = predictions(text)
answer = get_final_output(pred, unique_intent)
print(answer)

['i', 'would', 'like', 'to', 'get', 'some', 'technical', 'assistance']


C:\Users\rosha\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\keras\engine\sequential.py:425: UserWarning: `model.predict_proba()` is deprecated and will be removed after 2021-01-01. Please use `model.predict()` instead.
  warnings.warn('`model.predict_proba()` is deprecated and '


Once COVID AWARENESS App is set up for your Id, all you need to do scan the code which takes them to a unique page for your business. They then submit their contact information and answer the screening questions based on local legislation. Once completed they receive a green check mark to show you they have completed the process.


### Chatbot UI

In [17]:
from tkinter import *

In [19]:
def send():
    msg = EntryBox.get("1.0",'end-1c').strip()
    EntryBox.delete("0.0",END)
    # Adding Msg to screen, Clearing the Msg Box and Adding the reply to the chatbot from Model used for Prediction
    if msg != '':
        ChatLog.config(state=NORMAL)
        ChatLog.insert(END, "You: " + msg + '\n\n')
        ChatLog.config(foreground="#442265", font=("Verdana", 12 ))

        pred = predictions(msg)
        res = get_final_output(pred, unique_intent)
        
        ChatLog.insert(END, "Bot: " + res + '\n\n')

        ChatLog.config(state=DISABLED)
        ChatLog.yview(END)


base = Tk()
# Chat Window Dimensions
base.title("Durham College COVID-19 Assistance - Chat Bot")
base.geometry("400x500")
base.resizable(width=FALSE, height=FALSE)

# Creating Chat window
ChatLog = Text(base, bd=0, bg="white", height="8", width="50", font="Arial",)

ChatLog.config(state=DISABLED)

# Adding scrollbar to Chat window
scrollbar = Scrollbar(base, command=ChatLog.yview, cursor="heart")
ChatLog['yscrollcommand'] = scrollbar.set

# Creating Button to send message
SendButton = Button(base, font=("Verdana",12,'bold'), text="Send", width="12", height=5,
                    bd=0, bg="#33AAFF", activebackground="#3c9d9b",fg='#ffffff',
                    command= send )

# Creating the message-box to enter a message
EntryBox = Text(base, bd=0, bg="white",width="29", height="5", font="Arial")


# Adding all components on the screen
scrollbar.place(x=376,y=6, height=386)
ChatLog.place(x=6,y=6, height=386, width=370)
EntryBox.place(x=128, y=401, height=90, width=265)
SendButton.place(x=6, y=401, height=90)

base.mainloop()

['hello', 'i', 'need', 'some', 'information']
['what', 'is', 'contact', 'tracing']
['hey', 'how', 'are', 'you']
